# **University Life in Udine**

Are you a student wishing to study in Udine? Here's everything you need to know.

### **Some info about the notebook**

In this notebook there will be content designed specifically for users, others for programmers.

This distinction is made explicit by the use of the following icons:
- User 🧍‍♂️
- Programmer 👨‍💻

TODO: inserire qui la spiegazione generale del notebook.

---

## **Setup** 👨‍💻 

We import the necessary libraries for the notebook to correctly work.

In [7]:
import gpxpy
import pandas as pd
from datetime import datetime,timezone,timedelta
import geopandas as gpd
from geopy.geocoders import Nominatim
import movingpandas as mpd
import contextily as ctx
import folium
import leafmap
import os
import sys

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Import custom modules
sys.path.insert(0, os.path.abspath('../')) # need to pass absolute path in nb

print("> Setup Completed")

> Setup Completed


---

### Functions

TODO: move them to utility .py file

In [8]:
def read_gpx(path):
    """
    returns a gpx object, given its local path
    """
    gpx_file = open(path, 'r', encoding='UTF-8')
    return gpxpy.parse(gpx_file)


def print_gpx_info(gpx_file):
    """
    prints info about creator and structure of gpx file
    """
    for count_t, track in enumerate(gpx_file.tracks):
        print("Creator:", gpx_file.creator)
        #print("\nTracks, Segments and Points")
        print("> Track", count_t)
        for count_s, segment in enumerate(track.segments):
            print("  > Segment", count_s, "has", len(segment.points), "points")


def create_geodf_from_segment(gpx_file, track_idx, segment_idx):
    """
    Input
        > gpx_file
        > track_idx     index of track (first layer)
        > segment_idx   index of segment (second layer)

    Output
        > pandas dataframe with information of required segment
    """
    data = []
    for point_idx, point in enumerate(gpx_file.tracks[track_idx].segments[segment_idx].points):
        data.append([
            point.longitude, 
            point.latitude,
            point.elevation,
            point.time]
            ) 

    columns_name = ['longitude', 'latitude', 'altitude', 'time'] 
    gpx_dataframe = pd.DataFrame(data, columns=columns_name)

    gpx_dataframe['time'] = gpx_dataframe['time'].apply(lambda x: x.replace(tzinfo=None))

    geo_df = gpd.GeoDataFrame(
        gpx_dataframe, 
        crs = 4326,
        geometry = gpd.points_from_xy(gpx_dataframe.longitude, gpx_dataframe.latitude, gpx_dataframe.altitude)
        )

    return geo_df


def get_travel_time(df):
    """
    Given a geodf with a 'time' column, it returns the total time required (as html)
    Note: the 'time' column MUST be sorted in ascending order
    """
    # obtain start and end times
    start = df["time"][0]
    end = df["time"][len(df["time"])-1]

    # get travel time
    travel_time = end - start
    travel_time_hours = str(timedelta(seconds=travel_time.seconds))

    # show res
    print_travel_time = travel_time_hours.split(':')
    return "<b>Time</b>: %sh %sm" % (print_travel_time[0], print_travel_time[1])


def get_altitude(df):
    """
    Given a geodf with an 'altitude' column, it returns the min and max value present (as html)
    """
    min_alt = min(df["altitude"])
    max_alt = max(df["altitude"])
    return "<b>Minimum Altitude</b>: %s meters<br><b>Max Altitude</b>: %s meters" % (min_alt, max_alt)


def get_start_end_locations(df):
    """
    Given a geodf with a 'geometry' column, it returns the first and last location (as html) 
    """

    # setup Nominatin
    geolocator = Nominatim(user_agent="udine_project")

    # obtain lat and lon for start point point
    lat_start = df["geometry"][0].y
    lon_start = df["geometry"][0].x
    latlon_start = str(lat_start) + "," + str(lon_start)

    # obtain lat and lon for end point
    last_idx = len(df["geometry"]) - 1
    lat_end = df["geometry"][last_idx].y
    lon_end = df["geometry"][last_idx].x
    latlon_end = str(lat_end) + "," + str(lon_end)

    # obtain info and print them
    start_location = geolocator.reverse(latlon_start)
    end_location = geolocator.reverse(latlon_end)

    return "<b>Start Location</b>: %s <br><b>End Location</b>: %s" % (start_location[0], end_location[0])


def extract_lat_lon_for_folium(geodf):
    """
    Given a geodf with a 'geometry' columns, 
    it returns a list of tuples with all the values of lat/lon
    """
    coords = []
    for idx,row in geodf.iterrows():
        lat = row["geometry"].y
        lon = row["geometry"].x
        coords.append( (lat,lon) )
    return coords
    

## **Explore Routes**

First we explore the routes we have obtained thanks to [mapstogpx.com/strava](https://mapstogpx.com/strava/).

### **Parco del Cormor**

*Running route*

In [9]:
# import gpx file
path = "../data/strava/udine_run_parco_cormor.gpx"
udine_run_cormor = read_gpx(path)

# and show info about tracks, segments and points
print_gpx_info(udine_run_cormor)

Creator: https://www.mapstogpx.com/strava
> Track 0
  > Segment 0 has 754 points


In [10]:
# create dataframe from selected segment
udine_run_cormor_geodf = create_geodf_from_segment(udine_run_cormor, 0, 0)
udine_run_cormor_geodf

,longitude,latitude,altitude,time,geometry
0,13.214740,46.072182,115.1,2021-12-26 17:46:01,POINT Z (13.21474 46.07218 115.10000)
1,13.214760,46.072172,115.1,2021-12-26 17:46:02,POINT Z (13.21476 46.07217 115.10000)
2,13.214810,46.072143,115.1,2021-12-26 17:46:04,POINT Z (13.21481 46.07214 115.10000)
3,13.214862,46.072117,115.2,2021-12-26 17:46:06,POINT Z (13.21486 46.07212 115.20000)
4,13.214917,46.072090,115.2,2021-12-26 17:46:08,POINT Z (13.21492 46.07209 115.20000)
...,...,...,...,...,...
749,13.214889,46.072088,115.2,2021-12-26 19:02:24,POINT Z (13.21489 46.07209 115.20000)
750,13.214764,46.072137,115.1,2021-12-26 19:02:28,POINT Z (13.21476 46.07214 115.10000)
751,13.214667,46.072192,115.1,2021-12-26 19:02:32,POINT Z (13.21467 46.07219 115.10000)
752,13.214551,46.072258,115.0,2021-12-26 19:02:37,POINT Z (13.21455 46.07226 115.00000)


## INTERACTIVE MAPS TEST

### FUNZIONI PRINCIPALE

In [11]:
def create_base_map(lat,lon,list_of_layers):

    # initialize folium map centered on coords (Udine)
    base_map = folium.Map(location=[lat,lon])

    # add multiple layers
    for layer in list_of_layers:
        folium.TileLayer(layer).add_to(base_map)

    # create layer control
    folium.LayerControl().add_to(base_map)

    # return map
    return base_map


def add_route_info_to_map(geodf, title, route_type, color, base_map):

    # setup icon
    if route_type == "run":
        icon = folium.features.CustomIcon('../images/icon_run.png', icon_size=(34,34))
    elif route_type == "bike":
        icon = folium.features.CustomIcon('../images/icon_bike.png', icon_size=(34,34))
    else:
        print("ERROR: incorrect type provided")
        return 0

    # extract coords from geodf
    coords = extract_lat_lon_for_folium(geodf)

    # plot route
    folium.PolyLine(coords,
                    color=color,
                    weight=1.2,
                    opacity=0.75).add_to(base_map)

    # obtain info about route
    travel_time = get_travel_time(geodf)
    altitude = get_altitude(geodf)
    start_end = get_start_end_locations(geodf)

    # prepare popup
    html = "<h3>" + title + "</h3>" + "<p>" + travel_time + "</p><p>" + altitude + "</p>" + "</p><p>" + start_end + "</p>"
    iframe = folium.IFrame(html=html, width=320, height=200)

    # add marker
    marker = folium.Marker(
        location=[coords[0][0],coords[0][1]],
        popup=folium.Popup(iframe, max_width=800),
        tooltip=title,
        icon=icon
    )
    marker.add_to(base_map)

    return base_map
    

In [13]:
# Create the base map

udine_lat = 46.0609604
udine_lon = 13.1980551
list_of_layers = ['Stamen Terrain','cartodbpositron']

base_map = create_base_map(udine_lat,udine_lon,list_of_layers)

# Add the routes to the basemap

# cormor
base_map = add_route_info_to_map(
    geodf = udine_run_cormor_geodf, 
    title = "Run in Parco del Cormor",
    route_type = "run", 
    color = "green", 
    base_map = base_map
    )

# save it as .html
base_map.save("../dataviz/interactive_map_for_running_and_cycling.html")

base_map